In [1]:
# First need to import the libraries and the data into a dataframe using pandas
import pandas as pd 
import numpy as np
# Seeing all the missing data
missing_values = ["?" or " "]
insurance_claim_data = pd.read_csv('./data/train.csv', na_values=missing_values, encoding= 'unicode_escape')
insurance_claim_data.shape
# Need to transform some of the variables from int64 to float64
for i in insurance_claim_data:
    if(insurance_claim_data[i].dtype == np.int64):
        insurance_claim_data[i] = insurance_claim_data[i].astype('float64')
for i in insurance_claim_data:    
    if(insurance_claim_data[i].dtype == np.float64):
        insurance_claim_data[i].fillna(insurance_claim_data[i].median(), inplace=True)
    else:
        insurance_claim_data[i].fillna(insurance_claim_data[i].mode()[0], inplace=True)
        
insurance_claim_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 35 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row_ID          30000 non-null  float64
 1   Household_ID    30000 non-null  float64
 2   Vehicle         30000 non-null  float64
 3   Calendar_Year   30000 non-null  float64
 4   Model_Year      30000 non-null  float64
 5   Blind_Make      30000 non-null  object 
 6   Blind_Model     30000 non-null  object 
 7   Blind_Submodel  30000 non-null  object 
 8   Cat1            30000 non-null  object 
 9   Cat2            30000 non-null  object 
 10  Cat3            30000 non-null  object 
 11  Cat4            30000 non-null  object 
 12  Cat5            30000 non-null  object 
 13  Cat6            30000 non-null  object 
 14  Cat7            30000 non-null  object 
 15  Cat8            30000 non-null  object 
 16  Cat9            30000 non-null  object 
 17  Cat10           30000 non-null 

<b> <Center> Data Preprocessing </Center> </b> <br>
(1a) <b>The dataset has several fields with missing data. Choose a method to deal with missing data and justify your choice</b> <br> <br>
Most features have a small percentage of missing data values. The missing values will be imputed by replacing them with the mode of the categorical variables and the median of the continuous variables. This method prevents the loss of data which would occur when removing the rows containing missing values. As a result, this method was chosen as it helps to retain a large dataset so that all the instances can be fairly represented when building the predictive model.<br>
The median will be used for continuous variables as unlike the mean this value cannot be skewed by outliers so it is more suitable for imputing the values for this dataset. With the mode used to replace the missing values for the categorical variables. This is excluding some of the categorical variables such as Cat2, Cat4, Cat5 and Cat7. These features contain a very large proportion of missing values (ranging from approximately 33-50%). As a result these features will be ignored to prevent the feature space from being too large.  An alternative would be to remove the rows containing missing values. However, this could result in the loss of a lot of data, especially for the test set which would hinder the prediction.

In [2]:
# Got rid of more variables as I couldn't fix the problem with different values in the training & test data
insurance_claim_data = insurance_claim_data.drop(['Cat1','Cat2', 'Cat4', 'Cat5', 'Cat7', 'Cat10', \
                                        'Cat11', 'Cat12', 'Blind_Submodel','Blind_Make','Blind_Model'], axis=1)
list(insurance_claim_data.columns.values)
insurance_claim_data.info()
insurance_claim_data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row_ID         30000 non-null  float64
 1   Household_ID   30000 non-null  float64
 2   Vehicle        30000 non-null  float64
 3   Calendar_Year  30000 non-null  float64
 4   Model_Year     30000 non-null  float64
 5   Cat3           30000 non-null  object 
 6   Cat6           30000 non-null  object 
 7   Cat8           30000 non-null  object 
 8   Cat9           30000 non-null  object 
 9   OrdCat         30000 non-null  float64
 10  Var1           30000 non-null  float64
 11  Var2           30000 non-null  float64
 12  Var3           30000 non-null  float64
 13  Var4           30000 non-null  float64
 14  Var5           30000 non-null  float64
 15  Var6           30000 non-null  float64
 16  Var7           30000 non-null  float64
 17  Var8           30000 non-null  float64
 18  NVCat 

(30000, 24)

In [3]:
 # Splitting the training set into training and testing data
from sklearn.model_selection import train_test_split
ic_train_set, ic_test_set = train_test_split(insurance_claim_data, test_size=0.15, random_state=42)

(1b) <b> Convert categorical values to a suitable representation. Notice that there are many categorical variables in the dataset. 
If you use all the categorical variables you will end up with a large feature space. 
Feel free to ignore categorical variables that will increase your feature space considerably 
but use at least five categorical variables </b> <br> <br>

OneHotEncoder() can be used to transform the categorical variables to a one-hot encoding representation which is suitable as it gives a higher-dimensional binary representation for each value.



In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
features_cat = ['Cat3', 'Cat8', \
                 'Cat9', 'Cat6', 'NVCat']
features_num = ['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', \
                'Model_Year', 'Var1', 'Var2', 'Var3','Var4', 'Var5', \
                'Var6', 'Var7', 'Var8', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4','OrdCat']
full_transform = ColumnTransformer([
    ("num", StandardScaler(), features_num),
    ("cat", OneHotEncoder(handle_unknown= 'ignore'), features_cat),
])
full_transform
# handle_unknown= 'ignore'

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['Row_ID', 'Household_ID', 'Vehicle',
                                  'Calendar_Year', 'Model_Year', 'Var1', 'Var2',
                                  'Var3', 'Var4', 'Var5', 'Var6', 'Var7',
                                  'Var8', 'NVVar1', 'NVVar2', 'NVVar3',
                                  'NVVar4', 'OrdCat']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Cat3', 'Cat8', 'Cat9', 'Cat6', 'NVCat'])])

 <b> <Centered_text> (1c) The data is highly imbalanced: more records contain zero claims than not. 
    When designing your predictive model, you need to account for this </Centered_text> </b>
    
To account for this I can use undersampling of the zero claims inorder to balance the data for the training set. Undersampling can be used as there is already a large data sample so instances from the over-represented zero claims class can be ignored. This will help the predictive model perform better when the claim amount is not zero.

In [5]:
# copy the orginal dataset 
np.count_nonzero(ic_train_set['Claim_Amount']) 
ic_train_set.shape# The number of non zero claims
# Used this value and the total number of claims to find the fraction of zero claims
# needed to balance the data in the training set
# Balanced the data using undersampling
ic_train_set_sampled = ic_train_set.copy().drop(ic_train_set.query('Claim_Amount == 0').sample(frac=0.57).index)
ic_train_set_sampled['Claim_Amount']
ic_train_set_sampled.sample(10)
ic_train_set_sampled
# Before applying the transformation need to separate the target feature from the attributes
ic_train_set_attributes = ic_train_set_sampled.drop('Claim_Amount', axis=1)
ic_train_set_labels = ic_train_set_sampled['Claim_Amount']
# Now apply and fit the transformer to the training data
ic_train_set_attributes_prepared = full_transform.fit_transform(ic_train_set_attributes)

In [6]:
# Create a further training and validation set from the original training set
ic_train2_set, ic_val_set = train_test_split(ic_train_set, test_size=0.15, random_state=42)
# Balancing the training data
ic_train2_set_sampled = ic_train2_set.copy().drop(ic_train2_set.query('Claim_Amount == 0').sample(frac=0.57).index)
ic_train2_set_sampled['Claim_Amount']
ic_train2_set_sampled.sample(10)

# These are just Xtrain, ytrain, Xtest, ytest
ic_train2_set_attributes = ic_train2_set_sampled.drop('Claim_Amount', axis=1)
ic_train2_set_labels = ic_train2_set_sampled['Claim_Amount']
ic_val_set_attributes = ic_val_set.drop('Claim_Amount', axis=1)
ic_val_set_labels = ic_val_set['Claim_Amount']
# Fit tranform attributes in the training set and transform them in the validation set
ic_train2_set_attributes_transformed = full_transform.fit_transform(ic_train2_set_attributes)
ic_val_set_attributes_transformed = full_transform.transform(ic_val_set_attributes)

<b> <Center> Performance using a single model </Center>  <br>
(2a) Compute the performance of Linear Regression using gridsearch with at least 3 options for each parameter. 
Reporting the performance over the validation set </b>

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
# Picking the parameters for gridsearch
param_grid = {'fit_intercept':[True, False], 'normalize':[True, False], 'copy_X':[True, False]}
grid = GridSearchCV(LinearRegression(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')

# Scoring only given as negative mean squared error
# So need to get rid of the negative & then sqrt it to get rmse

# grid.fit(ic_train_set_attributes_prepared, ic_train_set_labels)
# grid.best_params_

In [8]:
# Found the best parameters
linear = LinearRegression(copy_X=True, fit_intercept=True, normalize=False)
linear.fit(ic_train2_set_attributes_transformed, ic_train2_set_labels)
ic_val_set_predictions = linear.predict(ic_val_set_attributes_transformed)
# Now calcuating the rmse
from sklearn.metrics import mean_squared_error
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
# 226.62

226.55888465026382

<b> (2b) Compute the performance of Ridge Regression using gridsearch with at least 3 options for each parameter. 
Reporting the performance over the validation set </b>

In [9]:
from sklearn.linear_model import Ridge
param_grid = {'alpha':[0.0, 0.5, 1.0, 1.5, 2.0],'fit_intercept':[True, False], 'normalize':[True, False], 'copy_X':[True, False]}
grid2 = GridSearchCV(Ridge(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid2.fit(ic_train_set_attributes_prepared, ic_train_set_labels)
# grid2.best_params_

In [10]:
# Found the best parameters
ridge = Ridge(alpha=1.0, copy_X=True, fit_intercept=True, normalize=True)
ridge.fit(ic_train2_set_attributes_transformed, ic_train2_set_labels)
ic_val_set_predictions = ridge.predict(ic_val_set_attributes_transformed)
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
# 225.85

225.8271087877898

<b> (2c) Compute the performance of Random Forests for regression using gridsearch with at least 3 options for each parameter. 
Reporting the performance over the validation set </b>

In [11]:
from sklearn.ensemble import RandomForestRegressor
RF_transform = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown= 'ignore'), features_cat),
],remainder='passthrough')

# Fit transform in the training set & then transform in the validation set
ic_train2_set_attributes_RF_trans = RF_transform.fit_transform(ic_train2_set_attributes)
ic_val_set_attributes_RF_trans = RF_transform.transform(ic_val_set_attributes)

param_grid = {'n_estimators':[20, 50, 100, 200], 'max_samples':[500, 1000, 2000, 3000], 'max_depth':[3, 5, 10, 15]}
grid = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(ic_train2_set_attributes_RF_trans, ic_train2_set_labels)
# grid.best_params_


In [12]:
# Found the best parameters
forest = RandomForestRegressor(max_depth=3, max_samples=3000, n_estimators=200)
forest.fit(ic_train2_set_attributes_transformed, ic_train2_set_labels)
ic_val_set_predictions = forest.predict(ic_val_set_attributes_RF_trans )
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
# 227.85

226.65649025729712

<b> (2d) Compute the performance of Gradient tree boosting for regression using gridsearch with at least 3 options for each parameter. Reporting the performance over the validation set </b>

In [13]:
from sklearn.ensemble import GradientBoostingRegressor
param_grid = {'n_estimators':[20, 50, 100, 200], 'max_depth':[3, 5, 10, 15],
              'learning_rate':[0.1, 0.2, 0.3, 0.4]}
grid = GridSearchCV(GradientBoostingRegressor(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(ic_train2_set_attributes_RF_trans, ic_train2_set_labels)
# grid.best_params_

In [14]:
grad_boost = GradientBoostingRegressor(max_depth=3, learning_rate=0.1, n_estimators=20)
grad_boost.fit(ic_train2_set_attributes_transformed, ic_train2_set_labels)
ic_val_set_predictions = grad_boost.predict(ic_val_set_attributes_transformed)
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
# 228.05

227.40376332536425

Ridge regression performed the best with the lowest RMSE with the gradient boosting regressor performing the worst with the highest RMSE. However, all the single models performed very similarly with RMSE in the range of 225-228.

<b> <Center> Performance using a combination of two models </Center> <br>
(3a) The first model will be a binary classifier that will tell whether the claim was zero or different from zero.
Compare the following classifiers: random forests for classification and gradient boosting for classification </b>

In [15]:
# Forming a new binary variable assigning 1 as non zero and 0 as zero claims
# For training data
ic_train2_set_binary = ic_train2_set_sampled.copy()
ic_train2_set_binary.query('Claim_Amount == 0')
ic_train2_set_binary["Binary"] = 1
ic_train2_set_binary.loc[ic_train2_set_binary["Claim_Amount"] == 0, "Binary"] = 0
Xtrain2_binary = ic_train2_set_binary.drop('Claim_Amount', axis=1)
# Testing it works
ic_train2_set_binary.query('Claim_Amount != 0')
ic_train2_set_binary.query('Claim_Amount == 0')
# For validation data
ic_val_set_binary = ic_val_set.copy()
ic_val_set_binary["Binary"] = 1
ic_val_set_binary.loc[ic_val_set_binary["Claim_Amount"] == 0, "Binary"] = 0
Xval_binary = ic_val_set_binary.drop('Claim_Amount', axis=1)
# Testing it works
ic_train2_set_binary.query('Claim_Amount == 0')
ic_val_set_binary.query('Claim_Amount != 0')

# Now need to set the binary variable as y to make the classification
# Not sure whether Claim_Amount is in xtrain or ytrain now
Xtrain2 = Xtrain2_binary.iloc[:, :-1]
ytrain2 = Xtrain2_binary.iloc[:, -1:]
Xval = Xval_binary.iloc[:, :-1]
yval = Xval_binary.iloc[:, -1:]
# Transform 
Xtrain2_attributes = RF_transform.fit_transform(Xtrain2)
Xval_attributes = RF_transform.fit_transform(Xval)


In [16]:
from sklearn.ensemble import RandomForestClassifier
param_grid = {'criterion':['entropy', 'gini'], 'n_estimators':[20, 50, 100, 200], 'max_depth':[3, 5, 10, 15],}
grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(Xtrain2_attributes, ytrain2)
# grid.best_params_

In [17]:
forest_classifier = RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=50)
forest_classifier.fit(Xtrain2_attributes, ytrain2.values.ravel())
yval_predict = forest_classifier.predict(Xval_attributes)
error_mod = np.sqrt(mean_squared_error(yval, yval_predict))
error_mod
# 0.661

0.6607581306268901

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {'loss':['deviance', 'exponential'], 'n_estimators':[20, 50, 100, 200], 'max_depth':[3, 5, 10, 15],
             'criterion':['friedman_mse', 'mse', 'mae']}
grid = GridSearchCV(GradientBoostingClassifier(), param_grid=param_grid, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(Xtrain2_attributes, ytrain2)
# grid.best_params_


In [19]:
grad_classifier = GradientBoostingClassifier(criterion='friedman_mse', loss='exponential', n_estimators=100, max_depth=3)
grad_classifier.fit(Xtrain2_attributes, ytrain2.values.ravel())
yval_predict = grad_classifier.predict(Xval_attributes)
error_mod = np.sqrt(mean_squared_error(yval, yval_predict))
error_mod
# 0.653

0.6595700718942318

The Random forest classifer gave a slightly better performance with a marginally better accuracy when classifying whether the claims were zero or nonzero. The best hyperparameters were used for both instances.

<b> (3b) For the second model, if the claim was different from zero, train a regression model to predict the actual value of the claim. Compare the same models that you used in step 2 </b>

In [20]:
# Form the training set containing only non-zero values
ic_train2_nonzero = ic_train2_set_binary.loc[ic_train2_set_binary["Binary"] == 1].copy()
ic_train2_nonzero = ic_train2_nonzero.drop('Binary', axis=1)
Xtrain = ic_train2_nonzero.iloc[:, 0:23]
ytrain = ic_train2_nonzero.iloc[:, 23]
Xtrain_trans = full_transform.fit_transform(Xtrain)

# For LinearRegression    
param_grid = {'fit_intercept':[True, False], 'normalize':[True, False], 'copy_X':[True, False]}
grid = GridSearchCV(LinearRegression(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(Xtrain_trans, ytrain)
# grid.best_params_


In [21]:
linear_q3 = LinearRegression(copy_X=True, fit_intercept=True, normalize=False)
linear_q3.fit(Xtrain_trans, ytrain)
ic_val_set_predictions = linear_q3.predict(ic_val_set_attributes_transformed)
# Now calcuating the rmse
from sklearn.metrics import mean_squared_error
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
#268.57

268.43831184741634

In [22]:
# For Ridge Regression
param_grid = {'alpha':[0.0, 0.5, 1.0, 1.5, 2.0],'fit_intercept':[True, False], 'normalize':[True, False], 'copy_X':[True, False]}
grid = GridSearchCV(Ridge(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(Xtrain_trans, ytrain)
# grid.best_params_

In [23]:
ridge_q3 = Ridge(alpha=1.0, copy_X=True, fit_intercept=True, normalize=True)
ridge_q3.fit(Xtrain_trans, ytrain)
ic_val_set_predictions = ridge_q3.predict(ic_val_set_attributes_transformed)
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
#263.36

263.3442004025868

In [24]:
# For Random Forests for Regression

# Only need to transform the new Xtrain data containing only nonzeros
Xtrain_RF_trans = RF_transform.fit_transform(Xtrain)

param_grid = {'n_estimators':[20, 50, 100, 200], 'max_samples':[500, 1000, 2000, 3000], 'max_depth':[3, 5, 10, 15]}
grid = GridSearchCV(RandomForestRegressor(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(Xtrain_RF_trans, ytrain)
# grid.best_params_

In [25]:
forest_q3 = RandomForestRegressor(max_depth=3, max_samples=3000, n_estimators=200)
forest_q3.fit(Xtrain_RF_trans, ytrain.values.ravel())
ic_val_set_predictions = forest_q3.predict(ic_val_set_attributes_RF_trans)
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
#270.32

268.7952800237793

In [26]:
# For gradient tree boosting for regression
param_grid = {'n_estimators':[20, 50, 100, 200], 'max_depth':[3, 5, 10, 15],
              'learning_rate':[0.1, 0.2, 0.3, 0.4]}
grid = GridSearchCV(GradientBoostingRegressor(), param_grid=param_grid, verbose=1, n_jobs=-1, scoring='neg_mean_squared_error')
# grid.fit(Xtrain_RF_trans, ytrain)
# grid.best_params_

In [27]:
grad_boost_q3 = GradientBoostingRegressor(max_depth=3, learning_rate=0.1, n_estimators=20)
grad_boost_q3.fit(Xtrain_RF_trans, ytrain.values.ravel())
ic_val_set_predictions = grad_boost_q3.predict(ic_val_set_attributes_RF_trans)
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, ic_val_set_predictions))
error_mod
#270.38

270.38821128144593

The best model for predicting the non-zero claims was also the Ridge regression model
with the lowest root mean square error value. Linear Regression, Gradient tree boosting and random forest regressors
had very similar error values in there predictions with root mean square errors ranging from 268-270.
However, the Ridge regression model had a lower value of approximately 263. Once again, all the 
models performed very similarly which can be seen by the almost identical root mean square errors.

<b> (3c) Use the tandem model built from steps a and b, for predicting in the same validation data 
used in Step 2, and report the performance </b>

The gradient boosting classifier and the ridge regression model will be use to form the tandem model.
This is because they produced the best results in terms of accuracy for the classifier and rmse for the regression model.

In [28]:
# So need to use this on the training data containing zeros and nonzeros
# First classifier predicts whether zero or nonzero
# If nonzero then use ridge regression to predict the claim amount
grad_classifier_tandem = GradientBoostingClassifier(criterion='friedman_mse', loss='exponential', n_estimators=100, max_depth=3)
grad_classifier_tandem.fit(Xtrain2_attributes, ytrain2.values.ravel())
yval_predict = grad_classifier_tandem.predict(Xval_attributes)
ridge_tandem = Ridge(alpha=1.0, copy_X=True, fit_intercept=True, normalize=True)
ridge_tandem.fit(Xtrain_trans, ytrain)
yval_predictions = ridge_tandem.predict(ic_val_set_attributes_transformed)
predictions = np.array([])
c = 0
for prediction in yval_predict:
    if prediction == 1:
        pred = yval_predictions[c]
    elif prediction == 0:
        pred = 0
    c += 1
    predictions = np.append(predictions,pred)
error_mod = np.sqrt(mean_squared_error(ic_val_set_labels, predictions))
error_mod
# 241.94

241.71643091315218

 <b> <Center> Report the performance of the best models over the test set </Center>  <br>
 (4) Compute the performance metric over the test set for the best model in Step 2  and the best model in Step 3. </b>

The best model from step2 was the ridge regressor model, whilst the best model from step 3 was the tandem
model formed by combining the gradient boosting classifier and the ridge regressor.

In [29]:
# splitting the test data into X and Y
Xtest = ic_test_set.iloc[:, :-1]
ytest = ic_test_set.iloc[:, -1:]
ytest
Xtest_transformed = full_transform.transform(Xtest)
# step2 ridge regressor model
ridge_test = Ridge(alpha=1.0, copy_X=True, fit_intercept=True, normalize=True)
ridge_test.fit(ic_train_set_attributes_prepared, ic_train_set_labels)
ridgeTest_predictions = ridge_test.predict(Xtest_transformed)
error_mod = np.sqrt(mean_squared_error(ytest, ridgeTest_predictions))
error_mod
# 209.62


209.60946241051226

In [30]:
# need to form the binary variable for the train and test data
# train data
ic_train_set_binary = ic_train_set_sampled.copy()
ic_train_set_binary["Binary"] = 1
ic_train_set_binary.loc[ic_train_set_binary["Claim_Amount"] == 0, "Binary"] = 0
Xtrain_binary = ic_train_set_binary.drop('Claim_Amount', axis=1)
# Testing it works
ic_train_set_binary.query('Claim_Amount != 0')
ic_train_set_binary.query('Claim_Amount == 0')
# test data
ic_test_set_binary = ic_test_set.copy()
ic_test_set_binary["Binary"] = 1
ic_test_set_binary.loc[ic_test_set_binary["Claim_Amount"] == 0, "Binary"] = 0
Xtest_binary = ic_test_set_binary.drop('Claim_Amount', axis=1)
# Testing it works
ic_test_set_binary.query('Claim_Amount == 0')
ic_test_set_binary.query('Claim_Amount != 0')

# Now need to set the binary variable as y to make the classification
Xtrain_classifier = Xtrain_binary.iloc[:, :-1]
ytrain_classifier = Xtrain_binary.iloc[:, -1:]
Xtest_classifier = Xtest_binary.iloc[:, :-1]
ytest_classifier = Xtest_binary.iloc[:, -1:]

In [31]:
ic_train_nonzero = ic_train_set_binary.loc[ic_train_set_binary["Binary"] == 1].copy()
train = ic_train_nonzero.drop('Binary', axis=1)
Xtrain_binary = train.iloc[:, :-1]
ytrain_binary = train.iloc[:, -1:]
Xtrain_binary_trans = full_transform.fit_transform(Xtrain_binary)
Xtrain_attributes = RF_transform.fit_transform(ic_train_set_attributes)
Xtest_attributes = RF_transform.transform(Xtest_classifier)
Xtest_transformed.shape
tandem_test = ic_test_set['Claim_Amount']
tandem_test

2308     120.54790
22404      0.00000
23397      0.00000
25058      0.00000
2664       0.00000
           ...    
19691      0.00000
7752       0.00000
17465     10.90208
12912      0.00000
8714      97.14479
Name: Claim_Amount, Length: 4500, dtype: float64

In [32]:
grad_classifier_test = GradientBoostingClassifier(criterion='friedman_mse', loss='exponential', n_estimators=100, max_depth=3)
grad_classifier_test.fit(Xtrain_attributes, ytrain_classifier.values.ravel())
ytest_predict = grad_classifier_test.predict(Xtest_attributes)
ridge_test = Ridge(alpha=1.0, copy_X=True, fit_intercept=True, normalize=True)
ridge_test.fit(Xtrain_binary_trans, ytrain_binary)
ytest_predictions = ridge_test.predict(Xtest_transformed)
arr=np.array([])
c=0
for prediction in ytest_predict:
    if prediction == 1:
        pred=ytest_predictions[c]
    elif prediction == 0:
        pred = 0
    c += 1
    arr=np.append(arr,pred)
    
arr.shape
error_mod = np.sqrt(mean_squared_error(tandem_test, arr))
error_mod
#225.499

225.37055211970727

The ridge regression model performed the best on the test data with a lower RMSE than the tandem model.
However, the tandem model actually predicted the results with a greater accuracy by correctly predicting zero claim values so I will be using this model for predictions as the ridge regressor model was unable to predict zero claim values as successfully.

 <b> <Center> Present your solution </Center> <br>
   (5) Provide four interesting and meaningful observations/comments about your machine learning pipeline, 
    with minimum three sentences for each observation/comment. </b>


One surprising outcome of the pipeline is that the single models provided the best performance by having a lower root-mean-squared-error (RMSE). This is unexpected as initially I expected the tandem model to perform the best as it uses a classifier to distinguish between the zero and nonzero claim accounts before applying the regression model. However, the accuracy of the classifiers appears to be a limiting factor with an accuracy of only 65% when predicting whether the claim was zero or nonzero. However even on the test set the ridge regression model performed slightly better than the tandem model with an RMSE of 209 compared to 210. <br> <br>
Another notable observation is the similar performances of the regressions models on the validation data. There is not much difference between the models with RMSE with values ranging from 228 for the gradient boosting regressor to 225 for the ridge regression model. I expected a more notable difference between the linear models, for example I expected linear regression to have a higher value as I didn’t think the correlation would be precisely linear. <br> <br>
Furthermore, as previously mentioned the two classifiers performed very similarly on the data as well with a very small difference in their accuracies. This was unexpected as the trees are built in different ways so I expected one of the classifiers to be more suitable for this dataset. For example, the random forest decision trees are built independently using bagging whilst gradient boosting builds one tree at a time using boosting. This is important as gradient boosting combines the results of the trees throughout the process while random forest combines the results using the majority rules once the process has finished. As a result, I expected the difference in approaches of the classifiers to lead to differing accuracies in their predictions. <br> <br>
Additionally, the regression models performed worse in step 3b than in step 2. The difference between these instances was that in step 3b the models were only trained and focused on predicting nonzero values. This led to a worse performance than in step 2 where they were trained to provide predictions for the entire dataset of zero and nonzero claims. I believe the reasoning for this may simply just be because the models in step 3b were not trained to identify zero claim amounts and therefore struggled to predict these values in the validation set.



 <b> <Center> (6) Create a function that contains the best model you built from Steps 1 to 4 that we will use to 
assess the performance of your design over an independent test set </Center> </b> <br>

In [37]:
def my_insurance_claim_predictor(test_data):
    test_data = pd.read_csv('./data/test.csv', na_values=["?" or " "], encoding='unicode_escape')
    test_data
    for i in test_data:
        if(test_data[i].dtype == np.int64):
            test_data[i] = test_data[i].astype('float64')
    for i in test_data:
        if(test_data[i].dtype == np.float64):
            test_data[i].fillna(test_data[i].median(), inplace=True)
        else:
            test_data[i].fillna(test_data[i].mode()[0], inplace=True)
    test_data = test_data.drop(['Cat1','Cat2', 'Cat4', 'Cat5', 'Cat7', 'Cat10', \
                                            'Cat11', 'Cat12', 'Blind_Submodel','Blind_Make','Blind_Model'], axis=1)
    test_data_binary = test_data.copy()
    Xtest_data = test_data.drop('Claim_Amount', axis=1)
    Xtest_data
    Xtest_attributes = full_transform.transform(Xtest_data)
    test_data_binary["Binary"] = 1
    test_data_binary.loc[test_data_binary["Claim_Amount"] == 0, "Binary"] = 0
    Xtest_classifier = test_data_binary.drop('Claim_Amount', axis=1)
    Xtest_classifier = test_data_binary.iloc[:, 0:23]
    Xtest_classifier
    ytest_classifier = test_data_binary.iloc[:, -1:]
    ytest_classifier
    Xtest_class_trans = RF_transform.transform(Xtest_classifier)
        
    ytest_predict = grad_classifier_test.predict(Xtest_class_trans)
    ytest_predictions = ridge_test.predict(Xtest_attributes)
    prediction = np.array([])
    c = 0
    for predictions in ytest_predict:
        if predictions == 1:
            pred = ytest_predictions[c]
        elif predictions == 0:
            pred = 0
        c += 1
        prediction = np.append(prediction,pred)
    return prediction

In [38]:
Xtest = pd.read_csv('./data/test.csv', na_values=["?" or " "], encoding='unicode_escape')

In [39]:
prediction = my_insurance_claim_predictor(Xtest)
prediction

C:\Users\Euan\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


array([  0.        , 246.22225672, 157.41175612, 254.43972888,
       233.90515253, 160.99961676,   0.        , 171.47998238,
       186.11475579])